<a href="https://colab.research.google.com/github/ishancoderr/Data_preparetion_for_research/blob/master/GAN_RESEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [232]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [233]:
# Load the data from GitHub
data = pd.read_csv("https://raw.githubusercontent.com/ishancoderr/Data_preparetion_for_research/master/Lanslide_rainfall_data%20.csv")

In [234]:
data

,longitude,latitude,acc_rainfall,hours,Avg. Intensity mm/hr,landslide
0,6.519725,80.154021,316,7,45,yes
1,6.586835,80.160486,259,7,37,yes
2,6.093508,80.606877,119,4,30,yes
3,6.509504,80.231427,244,5,49,yes
4,6.260793,80.491387,237,6,40,yes
...,...,...,...,...,...,...
87,6.765405,80.868156,135,4,34,yes
88,6.965074,80.516389,316,7,45,yes
89,6.879253,80.442860,259,7,37,yes
90,6.848140,80.605650,119,4,30,yes


In [235]:
# Normalize the data to be between -1 and 1
data = (data - data.mean()) / data.std()

<ipython-input-235-e54b233d2492>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data = (data - data.mean()) / data.std()


In [236]:
data

,Avg. Intensity mm/hr,acc_rainfall,hours,landslide,latitude,longitude
0,0.382230,1.552824,1.241993,NaN,-1.738378,-1.241019
1,-0.671408,0.639521,1.241993,NaN,-1.716748,-1.059175
2,-1.593342,-1.603679,-0.913919,NaN,-0.223292,-2.395915
3,0.909049,0.399178,-0.195282,NaN,-1.479405,-1.268714
4,-0.276294,0.287018,0.523355,NaN,-0.609678,-1.942633
...,...,...,...,...,...,...
87,-1.066523,-1.347313,-0.913919,NaN,0.650850,-0.575313
88,0.382230,1.552824,1.241993,NaN,-0.526030,-0.034282
89,-0.671408,0.639521,1.241993,NaN,-0.772031,-0.266825
90,-1.593342,-1.603679,-0.913919,NaN,-0.227397,-0.351132


In [237]:
# Split the data into input and target variables
X = data[['acc_rainfall', 'hours', 'Avg. Intensity mm/hr']].values
data['landslide']=1


In [238]:
y=data['landslide']


In [239]:
# Define the generator network
def generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,3)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [240]:
# Define the discriminator network
def discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,1)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [241]:
# Compile the generator and discriminator
generator = generator()
discriminator = discriminator()

generator.compile(loss='mean_squared_error', optimizer='adam')
discriminator.compile(loss='mean_squared_error', optimizer='adam')

In [242]:
# Combine the generator and discriminator into a GAN
gan = tf.keras.Sequential([generator, discriminator])
gan.compile(loss='mean_squared_error', optimizer='adam')

In [243]:
# Train the GAN
for epoch in range(1000):
    # Train the discriminator
    X_fake = generator.predict(X)
     

    y_fake = np.zeros(len(y))
    X_real = X
    y_real = y

    discriminator.trainable = True
    discriminator.train_on_batch(X_real, y_real)
    discriminator.train_on_batch(X_fake, y_fake)

    # Train the generator
    discriminator.trainable = False
    gan.train_on_batch(X, y_real)

3/3 [==============================] - 0s 4ms/step


3/3 [==============================] - 0s 3ms/step


In [244]:
print(discriminator.evaluate(X_real, y_real, verbose=0))
print(discriminator.evaluate(X_fake, y_fake, verbose=0))

0.0416734553873539


0.4369909167289734


In [245]:
input_data = np.array([[10,5,2]])
input_data = (input_data - input_data.mean()) / input_data.std()

prediction = generator.predict(input_data)
prediction

1/1 [==============================] - 0s 37ms/step


array([[0.9999999]], dtype=float32)